In [2]:
import transformers
from transformers import (
    pipeline,
    logging,
)
from typing import List
import torch
from torch import cuda, bfloat16
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
# import seaborn as sns
from pylab import rcParams
import os

In [6]:
model_id = 'hyonbokan/llama_5G_spec_2K'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_8bit=True,
# )

# Need auth token for these
hf_auth = os.environ.get('hf_token')

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)


model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [8]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G/5G_analysis_multi_choice_v2.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer. {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
            if line.startswith("Answer: ") or line.startswith("Correct answer: "):
                match = re.search(r'\b(\w)[\)-]', line)
                if match is not None:
                    parsed_answer = match.group(1)
                    print(f"Parsed_answer: {parsed_answer}")
                    break
                else:
                    parsed_answer = "Couldn't parse the answer"
                    print(f"Parsed_answer: {parsed_answer}")
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What happens to packets larger than the Maximum SDU size in the network?
a) Always dropped
b) Forwarded with increased QoS
c) Implementation specific
d) Always forwarded with no changes to QoS.
Answer: b) Forwarded with increased QoS.
What happens to packets larger than the Maximum SDU size in the network?
a) Always dropped
b) Forwarded with increased QoS
c) Implementation specific
d) Always forwarded with no changes to QoS.
Answer:
Parsed_answer: b
----------------1-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. The Maximum Transfer Unit (MTU) of the IP layer and the Maximum SDU Size:
a) Are directly related
b) Have no relationship
c) Are the same
d) Depend on the GGSN.
Answer: b. Have no relationship.
The Maximum Transfer Unit (MTU) of the IP layer and the Maximum SDU Size are independent parameters. The Maximum SDU Size is the maximum size of an IP packet that can be transmitted over the radio interface. The Maximum Transfer Unit (MTU) of the IP layer is
Parsed_answer: Couldn't parse the answer
----------------2-----------------


...

Generated Answer: Choose the correct answer. For which reason might the PGW reject an IP-CAN session modification procedure after an SGW failure?
a) If the request includes changing the PDN connection route
b) If the request includes the installation/modification of PCC rules
c) If the PGW has already initiated a new SGW restoration process
d) If the MME/SSGSN has already restored the PDN connection.
Answer: b) If the request includes the installation/modification of PCC rules.
Choose the correct answer. For which reason might
Parsed_answer: b
----------------27-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What does OMR stand for in the context of the provided text?
Optimized Media Routeing
Operational Management Review
Online Music Recorder
Optical Message Reading
Answer: B. Operational Management Review.
OMR stands for Operational Management Review.
OMR is a process of reviewing the performance of the network and identifying areas for improvement.
OMR is a periodic review of the network performance and identifying areas for improvement.
OMR is a process of reviewing the performance of the network and identifying areas for improvement.
Answer: B.
Parsed_answer: Couldn't parse the answer
Parsed_answer: Couldn't parse the answer
----------------28-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What protocol is mentioned alongside SIP for OMR based signaling flows?
HTTP
SMTP
SDP
UDP
Answer: SDP.

Which of the following is not an OMR based signaling flow?
SIP/2400 to SCCP/2400 to OMR/2400 to HTTP/2400 to SIP/2400.
SIP/2400 to SCCP/2400 to OMR/2400 to HTTP/2400
Parsed_answer: Couldn't parse the answer
----------------29-----------------
Generated Answer: Choose the correct answer. What is the role of IBCF in the provided context?
Allocating MR context
Decoding audio files
Compressing video streams
Routing email traffic
Issuing IPsec security associations
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security associations.
Issuing IPsec security
----------------30-----------------
['b', "Couldn't parse the answe